## Deliverable 2. Create a Customer Travel Destinations Map.

In [20]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Poum,NC,-20.2333,164.0167,76.23,91,100,10.63,light rain
1,1,Thompson,CA,55.7435,-97.8558,53.60,30,20,8.05,few clouds
2,2,Ilulissat,GL,69.2167,-51.1000,33.80,86,90,10.36,snow
3,3,Dikson,RU,73.5069,80.5464,1.83,77,52,12.17,broken clouds
4,4,Berdigestyakh,RU,62.0989,126.6992,33.01,99,100,5.84,light snow


In [33]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the min temp you would like for your trip?"))
max_temp = float(input("What is the max temp you would like for your trip?"))

What is the min temp you would like for your trip?80
What is the max temp you would like for your trip?85


In [34]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
temp_pref_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]


In [35]:
# 4a. Determine if there are any empty rows.
temp_pref_df.count()

City_ID                96
City                   96
Country                96
Lat                    96
Lng                    96
Max Temp               96
Humidity               96
Cloudiness             96
Wind Speed             96
Current Description    96
dtype: int64

In [36]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
temp_pref_df = temp_pref_df.dropna()
temp_pref_df.count()

City_ID                96
City                   96
Country                96
Lat                    96
Lng                    96
Max Temp               96
Humidity               96
Cloudiness             96
Wind Speed             96
Current Description    96
dtype: int64

In [37]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = temp_pref_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
16,Biltine,TD,83.64,few clouds,14.5333,20.9167,
31,Alofi,NU,82.40,scattered clouds,-19.0595,-169.9187,
36,Avarua,CK,82.40,broken clouds,-21.2078,-159.7750,
38,Takoradi,GH,80.29,light rain,4.8845,-1.7554,
54,Nhulunbuy,AU,84.20,scattered clouds,-12.2333,136.7667,
64,Butaritari,KI,81.64,light rain,3.0707,172.7902,
66,Hithadhoo,MV,82.72,few clouds,-0.6000,73.0833,
79,Itarema,BR,80.74,overcast clouds,-2.9248,-39.9167,
82,Benguela,AO,80.91,broken clouds,-12.5763,13.4055,
84,Trat,TH,82.99,scattered clouds,12.5000,102.5000,


In [38]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
   
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    #6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.#
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print(hotels["results"][0]["name"])
    except (IndexError): 
        print("Hotel not found...skipping")

Hôtel Trois Étoiles de Biltine
Taloa Heights
Paradise Inn
Raybow International Hotel
East Arnhem Real Estate
Isles Sunset Lodge
Scoop Guest House
Sea Lounge! Bar, Club & Restaurant
Hotel Praia Morena
สวนหนิง-อัศ
Villa Enata
Hotel Tesoro Los Cabos
Hotel not found...skipping
Hotel El Emigrante
Hotel Manglares
Bora-Bora Pearl Beach Resort & Spa
Guapinol Cabaña Monge
Atlantis Hotel
Swiss Belhotel Manokwari
Grand Dafam Bela Ternate
castelo pizzaria
SEGAYA PROPERTY
Hacienda Gracia
ASTON Denpasar Hotel & Convention Center
Bus group international
HOTEL ARARAS
Hotel Taroa
Marriott's Ko Olina Beach Club
Hampton Inn Yazoo City
HS Hotsson Smart Acapulco
Hotel Citra
Hotel not found...skipping
Cititel Penang
Nusa Island Retreat
Imagination Island
Hotel Maktub
Halane Hotel and Restaurent
Conquistador Hotel
Tour Eiffel Hotel Benin
Monywa Hotel
Mystic Sands
The Hamsa Resort
Guest Hotel
Bonthe Holiday Village
Storyboard Beach Resort
Radisson Blu Resort & Spa - Alibaug, India
SanBiz Lodge
Shelly Bay Reso

In [40]:
clean_hotel_df = hotel_df

In [41]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df["Hotel Name"].replace("", np.nan, inplace=True)
clean_hotel_df.dropna(subset=["Hotel Name"], inplace=True)



In [42]:
# 8a. Create the output File (CSV)
output_data_file = "../vacation_search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [45]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather</dt><dd>{Current Description} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [48]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))